# Code for Remote Sensing Paper #

### 1 - Generate Backbone polygons ###

In [ ]:
import os

inShp    = r'C:\gwork\rs_dez\bakbone\osm_streams.shp'
breakPnt = r'C:\gwork\rs_dez\bakbone\pnts_break_osm.shp'
lnhsplit = r'C:\gwork\rs_dez\bakbone\osm_streams_v2.shp'
lnhdeadend = r'C:\gwork\rs_dez\bakbone\osm_streams_v3.shp'
outShp = r'C:\gwork\rs_dez\tsts\lnh_polygons.shp'
lnhidpnt = 'line_id'

conDB = {
    'HOST' : 'localhost', 'PORT' : '5432', 'USER' : 'postgres',
    'PASSWORD' : 'admin', 'TEMPLATE' : 'postgis_template'
}

from glass.g.prop.prj import get_shp_epsg

EPSG = get_shp_epsg(inShp)

# Create DB
from glass.ng.sql.db  import create_db
from glass.pys.oss import fprop

conDB["DATABASE"] = create_db(conDB, fprop(inShp, 'fn', forceLower=True))

# Send data to database
from glass.ng.sql.to import shp_to_psql

lnh_tbl = shp_to_psql(conDB, inShp, api="shp2pgsql", srsEpsgCode=EPSG)

# 1.1 - Get Points to break lines
from glass.g.gp.ovl.sql import line_intersection_pnt

break_pnt = line_intersection_pnt(conDB, lnh_tbl, "breakpnt_{}".format(lnh_tbl))

# 1.2 - Break Lines

from glass.g.to.shp import dbtbl_to_shp
break_pnt_shp = dbtbl_to_shp(conDB, break_pnt, os.path.join(
    os.path.dirname(outShp), break_pnt + '.shp'
), api="pgsql2shp", geom_col=geom, epsg=EPSG)

from glass.g.gop.brk import break_lines_on_points
breakLines = break_lines_on_points(inShp, break_pnt_shp, os.path.join(
    os.path.dirname(outShp), conDB["DATABASE"] + "_brked.shp"
), "OFID")

# 1.3 - Delete Deadends

from glass.g.gp.cln.sql import rm_deadend

lnh_tbl = shp_to_psql(conDB, breakLines, api="shp2pgsql", srsEpsgCode=EPSG)
lnh_tbl = rm_deadend(conDB, lnh_tbl, conDB["DATABASE"] + "_deadend")

# 1.4 - Produce Polygons

from glass.g.dp.cg.sql import lnh_to_polg

polygons = lnh_to_polg(conDB, lnh_tbl, fprop(outShp, 'fn', forceLower=True))

polygons_shp = dbtbl_to_shp(
    conDB, "polygons", outShp, api="psql", geom_col="geom",
    epsg=EPSG
)